In [27]:
import re
import markdown
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def create_google_doc(title, content):
    """
    Create a Google Doc with formatted content from markdown text.

    Args:
        title (str): The title for the new Google Doc
        content (str): Markdown-formatted content to convert

    Returns:
        str: Document ID if successful, None if failed
    """
    try:
        # Authenticate user
        auth.authenticate_user()

        # Build the Docs API service
        service = build('docs', 'v1')

        # Create new document
        doc = service.documents().create(body={'title': title}).execute()
        document_id = doc.get('documentId')
        print(f"Document created with ID: {document_id}")

        requests = []
        current_index = 1  # Start after title

        # Process content line by line
        lines = content.split("\n")
        for line in lines:
            if line.startswith("# "):  # H1
                requests.extend([
                    {
                        'insertText': {
                            'location': {'index': current_index},
                            'text': line[2:] + '\n'
                        }
                    },
                    {
                        'updateParagraphStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(line[2:]) + 1
                            },
                            'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                            'fields': 'namedStyleType'
                        }
                    }
                ])
                current_index += len(line[2:]) + 1

            elif line.startswith("## "):  # H2
                requests.extend([
                    {
                        'insertText': {
                            'location': {'index': current_index},
                            'text': line[3:] + '\n'
                        }
                    },
                    {
                        'updateParagraphStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(line[3:]) + 1
                            },
                            'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                            'fields': 'namedStyleType'
                        }
                    }
                ])
                current_index += len(line[3:]) + 1

            elif line.startswith("### "):  # H3
                requests.extend([
                    {
                        'insertText': {
                            'location': {'index': current_index},
                            'text': line[4:] + '\n'
                        }
                    },
                    {
                        'updateParagraphStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(line[4:]) + 1
                            },
                            'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                            'fields': 'namedStyleType'
                        }
                    }
                ])
                current_index += len(line[4:]) + 1

            elif line.startswith("- [ ]"):  # Checkbox
                text = line[6:]
                requests.extend([
                    {
                        'insertText': {
                            'location': {'index': current_index},
                            'text': text + '\n'
                        }
                    },
                    {
                        'createParagraphBullets': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(text) + 1
                            },
                            'bulletPreset': 'BULLET_CHECKBOX'
                        }
                    }
                ])
                mention_start = text.find("@")
                while mention_start != -1:
                    mention_end = text.find(" ", mention_start)
                    if mention_end == -1:
                        mention_end = len(text)  # Until the end of the line

                    # Calculate the exact range in the document
                    mention_doc_start = current_index + mention_start
                    mention_doc_end = current_index + mention_end

                # Add text styling for mentions
                    requests.append({
                        'updateTextStyle': {
                            'range': {
                                'startIndex': mention_doc_start,
                                'endIndex': mention_doc_end
                            },
                            'textStyle': {
                                'bold': True,
                                'foregroundColor': {
                                    'color': {
                                        'rgbColor': {
                                            'red': 0.0,  # Blue color
                                            'green': 0.0,
                                            'blue': 1.0
                                        }
                                    }
                                }
                            },
                            'fields': 'bold,foregroundColor'
                        }
                    })

                    # Find the next @mention
                    mention_start = text.find("@", mention_end)

                current_index += len(text) + 1

            elif line.startswith(("- ", "* ")):  # Bullet points
                indent_level = len(line) - len(line.lstrip())
                text = line

                requests.extend([
                    {
                        'insertText': {
                            'location': {'index': current_index},
                            'text': text + '\n'
                        }
                    }
                ])

                if indent_level > 0:
                    requests.append({
                        'updateParagraphStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(text) + 1
                            },
                            'paragraphStyle': {
                                'indentStart': {
                                    'magnitude': 36 * (indent_level // 2),
                                    'unit': 'PT'
                                }
                            },
                            'fields': 'indentStart'
                        }
                    })

                current_index += len(text) + 1

            elif line.startswith("---"):
              continue # Footer separator


            elif "Meeting recorded by:" in line or "Duration:" in line:  # Footer information
                requests.extend([
                    {
                        'insertText': {
                            'location': {'index': current_index},
                            'text': line + '\n'
                        }
                    },
                    {
                        'updateParagraphStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(line) + 1
                            },
                            'paragraphStyle': {
                                'namedStyleType': 'SUBTITLE',
                                'spaceAbove': {'magnitude': 10, 'unit': 'PT'},
                                'spaceBelow': {'magnitude': 10, 'unit': 'PT'},
                                'borderTop': {
                                    'width': {'magnitude': 1, 'unit': 'PT'},
                                    'padding': {'magnitude': 4, 'unit': 'PT'},
                                    'dashStyle': 'SOLID'
                                }
                            },
                            'fields': 'namedStyleType,spaceAbove,spaceBelow,borderTop'
                        }
                    },
                    {
                        'updateTextStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(line) + 1
                            },
                            'textStyle': {
                                'italic': True,
                                'foregroundColor': {
                                    'color': {
                                        'rgbColor': {
                                            'red': 0.4,
                                            'green': 0.4,
                                            'blue': 0.4
                                        }
                                    }
                                }
                            },
                            'fields': 'italic,foregroundColor'
                        }
                    }
                ])
                current_index += len(line) + 1

            else:  # Regular text
                if line.strip():  # Only process non-empty lines
                    requests.append({
                        'insertText': {
                            'location': {'index': current_index},
                            'text': line + '\n'
                        }
                    })
                    current_index += len(line) + 1

        # Apply all formatting
        if requests:
            service.documents().batchUpdate(
                documentId=document_id,
                body={'requests': requests}
            ).execute()

        return document_id

    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

# Example markdown content
markdown_content = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

# Create the document
title = "Product Team Sync Notes"
document_id = create_google_doc(title, markdown_content)
if document_id:
    print(f"Successfully created document with ID: {document_id}")


Document created with ID: 1fiHuAsLNLDNWFnaY8HuevOtY4yzNDVWJeZtGj8gT31M
Successfully created document with ID: 1fiHuAsLNLDNWFnaY8HuevOtY4yzNDVWJeZtGj8gT31M
